We load data here, and confirm some basic data and see if it loaded correctly and it's complete

In [7]:
import pandas as pd
import numpy as np
white_wine = pd.read_csv('winequality-white.csv',sep=';')
red_wine= pd.read_csv('winequality-red.csv',sep=';')

In [ ]:
print(white_wine.info)

In [8]:
red_wine['type']=1
white_wine['type']=0
wines=red_wine.append(white_wine,ignore_index=True)

From here onwards, we start the proper modeling and building of the neural network.

The first step is preparing the training and the test set, in this case we use a holdout for the test equivalent to 33% of the total data. The random_state is set to 42 for consistent results.

In [13]:
from sklearn.preprocessing import StandardScaler
X=wines.iloc[:,0:12]
X=wines.drop('quality',axis=1)
Y=np.ravel(wines.quality)


X_train=StandardScaler().fit_transform(X)

Data is normalized in the following section to avoid giving some attributes bigger importance. This step is important when dealing with numeric data on several algorithms.

In [62]:
from keras import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold


seed=7
np.random.seed(seed)
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
for train, test in kfold.split(X,Y):
    model=Sequential()
    model.add(Dense(128,input_dim=12,activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='SGD',loss='mse',metrics=['mae'])
    model.fit(X_train[train],Y[train],epochs=100,verbose=0)
    y_pred=model.predict(X_train[test])
mse_value,mae_value=model.evaluate(X_train[test],Y[test],verbose=0)
r2_scor=r2_score(Y[test],y_pred)

In [61]:
print(mae_value,mse_value,r2_score(Y[test],y_pred))

0.5356801748275757 0.4731940261979423 0.37735931243664644


Build the neural network based on the design and the structure desired.
Must have at the very least one input layer, one hidden layer and one output layer.
Input must have the structure of the data to be received, in this case we have 12 attributes, which is the amount of neurons we use.
Output is only for the quality of the wine, and quality is in the range 0:10, so 10 is used.
Hidden layers are flexible and there is no "right" setup.

In [ ]:
from keras import Sequential

from keras.layers import Dense

model= Sequential() #Define the type of Neural network, Sequential networks move the data in a sequence from input->hidden->output
model.add(Dense(12,activation='relu',input_shape=(11,))) #Define the input layer, using "Dense" as method, and 'relu' as activation method
model.add(Dense(8,activation='relu')) #Define hidden layer 1
model.add(Dense(10,activation='tanh')) #Define hidden layer 2
model.add(Dense(14,activation='relu')) #Define hidden layer 3
model.add(Dense(10,activation='sigmoid')) #Define output layer

In [ ]:
model.output_shape

In [ ]:
model.summary()

In [ ]:
model.get_config()

In [ ]:
from keras.utils import to_categorical
y_binary = to_categorical(y_train)

model.get_weights()
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_binary,epochs=10,batch_size=1,verbose=1)

In [ ]:
y_pred=model.predict_classes(X_test)

#np.round(y_pred,1)

In [ ]:

y_pred[:10]

In [ ]:
y_test[:10]

In [ ]:
y_binary_test = to_categorical(y_test)
score=model.evaluate(X_test,y_binary_test,verbose=1)
print(score)

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score
confusion_matrix(y_test,y_pred)

In [ ]:
precision_score(y_test,y_pred,average="weighted")

In [ ]:
recall_score(y_test,y_pred,average="weighted")

In [ ]:
f1_score(y_test,y_pred,average="weighted")

In [ ]:
cohen_kappa_score(y_test,y_pred)

Conclusiones>

Ya casi 3 horas después de haber comenzado, creo que tengo la idea básica aprendida, aunque tuve que recurrir bastante a google para lograr definir el tipo de red y predicción como multi-clase.
En el caso de ejemplo inicial con el tipo de vino, una comparación binaria tradicional la precisión era excelente, pero ya al querer determinar la calidad del vino se complicó más de lo que hubiera querido. Tampoco estoy del todo familiarizado con Python debido a gran parte del máster ser en R, y me confundo en hacer operaciones de mezcla y separación, selección de atributos y otros, además de que los modelos tienen formato diferente en python, y hay que usar mucho numpy.
Forzar capas ocultas intermedias para probar los puntos inferiores mencionados y cambiar de función y usar tanh afectó la precisión final, en fin, es un ejercicio de práctica para entender los fundamentos básicos así que tampoco espero buscar 99.99% de precisión, pero solo 57.61% máximo me hace preguntarme si realmente todo se hizo como se debía.

La parte de los parámetros de las capas se me quedó un poco difusa hasta que fui a Google, realmente los nombres usados en la función no son exactamente claros para alguien aprendiendo, incluso llegué a pensar que el 1 del output layer era porque se daba una única solución y no porque era en el rango 0,1. Luego de estar dando vueltas al código fue que logré entender que son necesarias múltiples neuronas en la capa de salida para dar valores superiores a 1, y fue que pude hacer la parte de clasificación multiclase.